# Homework 3

1\. Add shifts to the QR algorithm

Instead of factoring $A_k$ as $Q_k R_k$ (the way the pure QR algorithm without shifts does), the shifted QR algorithms:

i. Get the QR factorization $$A_k - s_k I = Q_k R_k$$
ii. Set $$A_{k+1} = R_k Q_k + s_k I$$

Choose $s_k = A_k(m,m)$, an approximation of an eigenvalue of A. 

The idea of adding shifts to speed up convergence shows up in many algorithms in numerical linear algebra (including the power method, inverse iteration, and Rayleigh quotient iteration).   

In [59]:
import numpy as np

In [81]:
max_diff = 1e-32

In [134]:
def pure_qr(A):

    Qk,Rk = np.linalg.qr(A)
    Ak = Rk @ Qk
    Q = Qk

    sk = Ak[-1,-1]
    diff = 1
    iters = 0

    while diff > max_diff:
    # while iters < 30000:
        Qk,Rk = np.linalg.qr(Ak)
        Ak = Rk @ Qk
        Q = Q @ Qk

        diff = abs(sk - Ak[-1,-1])
        sk = Ak[-1,-1]
        iters += 1

        # if iters % 200 == 0:
        #     print(iters, sk, Ak[-1,-1], diff)

    return Ak, Q, iters

In [135]:
def practical_qr(A):

    n = A.shape[0]
    refeig = -1
    shift_ind = -1

    Qk,Rk = np.linalg.qr(A)
    Ak = Rk @ Qk
    Q = Qk

    sk = Ak[refeig,refeig]
    diff = 1
    iters = 0

    while diff > max_diff:
    # while iters < 30000:
        Qk,Rk = np.linalg.qr(Ak - Ak[shift_ind,shift_ind] * np.eye(n))
        Ak = Rk @ Qk + Ak[shift_ind,shift_ind] * np.eye(n)
        Q = Q @ Qk

        diff = abs(sk - Ak[refeig,refeig])
        sk = Ak[refeig,refeig]
        iters += 1

        # if iters % 200 == 0:
        #     print(iters, sk, Ak[refeig,refeig], diff)

    return Ak, Q, iters

In [138]:
n = 4
A = np.random.rand(n,n)

In [139]:
Ak, Q, iters = practical_qr(A)

Check that Q is orthogonal:

In [140]:
np.allclose(np.eye(n), Q @ Q.T), np.allclose(np.eye(n), Q.T @ Q)

(True, True)

Diagonals of $A_k$ vs eigenvalues of $A$:

In [141]:
np.diagonal(Ak), np.linalg.eigvals(A)

(array([ 1.95150665, -0.45771427,  0.32476741,  0.32476741]),
 array([ 1.95150665+0.j        ,  0.32476741+0.42641985j,
         0.32476741-0.42641985j, -0.45771427+0.j        ]))

Comparing convergence to pure QR:

In [142]:

Ak_pure, Q_pure, iters_pure = pure_qr(A)

In [143]:
print(np.linalg.eigvals(A))
print(np.diagonal(Ak), iters) 
print(np.diagonal(Ak_pure), iters_pure) 

[ 1.95150665+0.j          0.32476741+0.42641985j  0.32476741-0.42641985j
 -0.45771427+0.j        ]
[ 1.95150665 -0.45771427  0.32476741  0.32476741] 54
[ 1.95150665  0.36702199  0.28251282 -0.45771427] 216
